In [1]:
#Imports
from comet_ml import Experiment
experiment = Experiment(api_key="YkPEmantOag1R1VOJmXz11hmt", parse_args=False, project_name='SegNet')

COMET INFO: old comet version (1.0.44) detected. current: 1.0.55 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET WARNING: Failing to collect the installed os packages
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/yikeqicn/segnet/6761d4809cce41f6a12a51298e801e07



In [2]:
from os.path import join, basename, dirname
import tensorflow as tf
import shutil
import os
import sys
import argparse

import cv2
import numpy as np
import os
import time

from torch.utils.data import DataLoader, ConcatDataset, random_split#, SequentialSampler #yike: add SequentialSampler
import torch
import torchvision
import torchvision.transforms as transforms
from datasets import ArtTypeWriter
from Model_Unet_github import *

home = os.environ['HOME']

In [3]:
#General Settings
parser = argparse.ArgumentParser()

# system basics
parser.add_argument("-name", default='segnet_unet_lonly_typewriter_57000_200', type=str, help="name of the log") #debug model_intersect # segnet_no_intersect_1conv_64_channels_30epoch_unet_github
parser.add_argument("-gpu", default='1', type=str, help="gpu numbers")

parser.add_argument("-train", default=True, help="train the NN", action="store_true")

parser.add_argument("-transfer",default=False, help="test the NN", action="store_true")

parser.add_argument("-test",default=False, help="test the NN", action="store_true")

# image and logistic parameters 
parser.add_argument("-image_h", default=32, type=int, help='image height') #('image_h', "360", """ image height """) 32
parser.add_argument("-image_w", default=128, type=int, help='image width')#('image_w', "480", """ image width """)128
#parser.add_argument("-image_h", default=360, type=int, help='image height') 
#parser.add_argument("-image_w", default=480, type=int, help='image width')

parser.add_argument("-image_c", default=1, type=int, help='image channel')#('image_c', "3", """ image channel (RGB) """)
parser.add_argument("-num_class", default=2, type=int, help='total class number')

# training hyperparam
parser.add_argument("-batch_size", default=10, type=int, help='batch_size')
parser.add_argument("-lrInit", default=1e-3, type=int, help='initial lr')
parser.add_argument("-lrDrop1", default=10, type=int, help='step to drop lr by 10 first time') # not sure
parser.add_argument("-lrDrop2", default=1000, type=int, help='step to drop lr by 10 sexond time') # not sure
parser.add_argument('-max_epoch',default=200, type=int,help='max epoch numbers')



# file paths
parser.add_argument('-ckpt_root', default="/root/ckpt", type=str,help= "dir to store ckpt") # log_dir !!!!!
parser.add_argument('-data_root', default="/root/datasets", type=str, help=" root to any data folder ")
parser.add_argument('-urlTranferFrom', default="", type=str, help=" archived model url ")


#args = parser.parse_args()
args = parser.parse_known_args()[0]

name = args.name

experiment.set_name(name)
experiment.log_parameters(vars(args))

os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

ckptroot = args.ckpt_root
args.ckptpath = join(ckptroot, name)
if args.name=='debug': shutil.rmtree(args.ckptpath, ignore_errors=True)
os.makedirs(args.ckptpath, exist_ok=True)


In [4]:
def checkArgs():
    if args.test:
        print('The model is set to Testing')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid testing dir: %s"%FLAGS.test_dir)
    elif args.transfer:
        print('The model is set to transfer learn from ckpt')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Image dir: %s"%FLAGS.image_dir)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)
    else:
        print('The model is set to Training')
        print("Max training Iteration: %d"%args.max_epoch)
        print("Initial lr: %f"%args.lrInit)
        print("First Drop Steps: %i"%args.lrDrop1)
        print("Second Drop Steps: %i"%args.lrDrop2)
        print("Data root: %s"%args.data_root)
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)

    print("Batch Size: %d"%args.batch_size)

In [5]:
#main function
def main():    
    checkArgs()
    if args.train:
        transform_train = transforms.Compose([
          transforms.Lambda(lambda img: cv2.resize(img, (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC)),
          transforms.Lambda(lambda img: np.expand_dims(img,3) ),
          #transforms.Lambda(lambda img: add_artifacts(img,args)),
          #transforms.Lambda(lambda img: cv2.transpose(img))
        ])
        #arprint=ArtPrint(args.data_root, transform=transform_train)
        arprint=ArtTypeWriter(args.data_root, transform=transform_train)
        concat=arprint
        print(len(arprint))
        idxTrain = int( len(arprint)*0.9)
        trainset, testset = random_split(concat, [idxTrain, len(concat)-idxTrain])
        trainloader = DataLoader(trainset, batch_size=args.batch_size, shuffle=True, drop_last=True,num_workers=4)
        testloader = DataLoader(testset, batch_size=args.batch_size, shuffle=False, drop_last=False,num_workers=2)
        
        #weight gen
        pos_perc=sum(map(lambda x: np.sum(cv2.imread(x[1],cv2.IMREAD_GRAYSCALE)),trainset.dataset.samples))/(args.image_h*args.image_w*len(trainset.dataset.samples))
        neg_perc=1-pos_perc
        weight=np.array([pos_perc,neg_perc]) # just reverse
        print(weight)
        model=Model(args, experiment, loss_weight=weight, mustRestore=False)
        model.training(loader=trainloader, validateloader=testloader)
        
    else:
        pass # for now, leave it pass 
    

In [6]:
main()

The model is set to Training
Max training Iteration: 200
Initial lr: 0.001000
First Drop Steps: 10
Second Drop Steps: 1000
Data root: /root/datasets
Check point file: /root/ckpt/segnet_unet_lonly_typewriter_57000_200
Batch Size: 10
57972
[0.05666649 0.94333351]
Instructions for updating:
Colocations handled automatically by placer.
GGG
[None, 32, 128, 1]
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


0 conv1: (?, ?, ?, 32)
0 conv2: (?, ?, ?, 32)
1 conv1: (?, ?, ?, 64)
1 conv2: (?, ?, ?, 64)
2 conv1: (?, ?, ?, 128)
2 conv2: (?, ?, ?, 128)
1 h_deconv: (?, ?, ?, 64)
1 h_deconv_concat: (?, ?, ?, ?)
1 h_conv1_post_deconv: (?, ?, ?, 64)
1 h_conv2_post_deconv: (?, ?, ?, 64)
0 h_deconv: (?, ?, ?, 32)
0 h_deconv_concat: (?, ?, ?, ?)
0 h_conv1_post_deconv: (?, ?, ?, 32)
0 h_conv2_post_deconv: (?, ?, ?, 32)
0 outmap: (?, ?, ?, 2)
(?, ?, ?, 2)
loss: ()
INFO:tensorflow:Summary name loss/cross_entropy (raw) is illegal; using loss/cross_entropy__raw_ instead.


...

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys


TRAIN: Batch: 0.0 Loss: 0.0024525232
accuracy = 0.988940
mean IU  = 0.907776
    class # 0 capture rate = 0.988475 
    class # 1 capture rate = 0.997241 
TRAIN: Batch: 0.019168104274487254 Loss: 0.002468157
accuracy = 0.987231
mean IU  = 0.895358
    class # 0 capture rate = 0.986829 
    class # 1 capture rate = 0.994444 
TRAIN: Batch: 0.03833620854897451 Loss: 0.0023604068
accuracy = 0.989721
mean IU  = 0.920625
    class # 0 capture rate = 0.989356 
    class # 1 capture rate = 0.995488 
TRAIN: Batch: 0.05750431282346176 Loss: 0.0032585263
accuracy = 0.985838
mean IU  = 0.886570
    class # 0 capture rate = 0.985406 
    class # 1 capture rate = 0.993551 
TRAIN: Batch: 0.07667241709794902 Loss: 0.0023977167
accuracy = 0.987793
mean IU  = 0.902402
    class # 0 capture rate = 0.987290 
    class # 1 capture rate = 0.996446 
TRAIN: Batch: 0.09584052137243626 Loss: 0.0029181866
accuracy = 0.985718
mean IU  = 0.882144
    class # 0 capture rate = 0.985111 
    class # 1 capture rate = 

TRAIN: Batch: 0.9392371094498754 Loss: 0.0033229284
accuracy = 0.984495
mean IU  = 0.872751
    class # 0 capture rate = 0.984318 
    class # 1 capture rate = 0.987840 
TRAIN: Batch: 0.9584052137243627 Loss: 0.001860817
accuracy = 0.993676
mean IU  = 0.952057
    class # 0 capture rate = 0.993421 
    class # 1 capture rate = 0.997362 
TRAIN: Batch: 0.9775733179988499 Loss: 0.0017727141
accuracy = 0.990381
mean IU  = 0.920459
    class # 0 capture rate = 0.990000 
    class # 1 capture rate = 0.996901 
TRAIN: Batch: 0.9967414222733372 Loss: 0.002717975
accuracy = 0.985375
mean IU  = 0.884619
    class # 0 capture rate = 0.984678 
    class # 1 capture rate = 0.997715 
Validating NN


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys


VALID: Total accuracy: 98.952577%. Class 0 capture: 98.922196%. Class 1 capture: 99.461827%
Character error rate improved, save model
Done with training at epoch 200 sigoptObservation=0.989525765225394
